# Input/Outpt (IO)
Once you've processed a text or corpus with medspaCy, a next step is often to save and analyze the information you've extracted. `medpsacy.io` contains utilities for reading documents, converting processed docs into structured data, and writing your results to disk or to a database.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.insert(0, "..")

In [4]:
import medspacy

In [5]:
import sqlite3

#If you haven't already, install this pre-trained [med7](https://github.com/kormilitzin/med7) model and `spacy-transformers`:

In [6]:
# !pip install https://github.com/abchapman93/spacy_models/raw/master/releases/en_info_3700_i2b2_2012-0.1.0/dist/en_info_3700_i2b2_2012-0.1.0.tar.gz
# !pip install spacy-transformers

In [7]:
enable = ['pyrush',
 'tagger',
 'parser',
 'ner',
 'target_matcher',
 'context',
 'sectionizer']
nlp = medspacy.load("en_core_med7_trf", enable=enable)

/Users/alecchapman/opt/anaconda3/envs/spacy3/lib/python3.8/site-packages/packaging/version.py:127: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(


In [8]:
nlp.pipe_names

['transformer',
 'ner',
 'medspacy_pyrush',
 'medspacy_target_matcher',
 'medspacy_context',
 'medspacy_sectionizer']

As an example, we'll use a very simple sqlite database containing two sample documents.

In [9]:
import sqlite3 as sqlite

In [10]:
from helpers import create_medspacy_demo_db

In [12]:
create_medspacy_demo_db(drop_existing=True)

Created file medspacy_demo.db


In [13]:
conn = sqlite.connect("medspacy_demo.db")

In [14]:
cursor = conn.cursor()

In [15]:
cursor.execute("SELECT text FROM texts;")

In [16]:
text = cursor.fetchone()[0]

In [17]:
print(text[:600])

Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]

Date of Birth:  [**2498-8-19**]             Sex:   F

Service: SURGERY

Allergies:
Hydrochlorothiazide

Attending:[**First Name3 (LF) 1893**]
Chief Complaint:
Abdominal pain

Major Surgical or Invasive Procedure:
PICC line [**6-25**]
ERCP w/ sphincterotomy [**5-31**]


History of Present Illness:
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain. Imaging shows no evidence of metastasis. She is not receiving any chemo.

Past Medical History:
1. Colon cance


In [18]:
conn.close()

In [19]:
doc = nlp(text)

# I. DocConsumer
The `DocConsumer` class takes the attributes extracted by medspaCy and converts them into structured data. There are four different data types that the `DocConsumer` will extract:
- **"ent"**: Extract information about the spans in `doc.ents`. Each row will represent a single entity and can include either native spaCy attributes (ie., `ent.label_`) or custom attrributes (ie., `ent._.is_negated`)
- **"section"**: Each row will represent a section of the document and includes attributes such as the section text and category
- **"context"**: This represents the entity-modifier pairs extracted by ConText
- **"doc"**: A single row for the entire doc. By default this will only include `doc.text`, but you can add other underscore attributes

Let's create a `DocConsumer` with all four of these data types. We'll use the default attributes for now but will show how to customize them later.

In [20]:
from medspacy.io import DocConsumer

In [21]:
doc_consumer = DocConsumer(nlp, dtypes=("ent", "context", "section", "doc"))

`dtype_attrs` maps the data types to the corresponding attributes/columns

In [22]:
doc_consumer.dtype_attrs

{'ent': ['text',
  'start_char',
  'end_char',
  'label_',
  'is_negated',
  'is_uncertain',
  'is_historical',
  'is_hypothetical',
  'is_family',
  'section_category',
  'section_parent'],
 'section': ['section_category',
  'section_title_text',
  'section_title_start_char',
  'section_title_end_char',
  'section_text',
  'section_text_start_char',
  'section_text_end_char',
  'section_parent'],
 'context': ['ent_text',
  'ent_label_',
  'ent_start_char',
  'ent_end_char',
  'modifier_text',
  'modifier_category',
  'modifier_direction',
  'modifier_start_char',
  'modifier_end_char',
  'modifier_scope_start_char',
  'modifier_scope_end_char'],
 'doc': ['text']}

Now when we process our doc, we can get the relevant information from the `doc._.data` attribute. This is a nested dictionary where the outermost keys are the data types, and for each data type is an ordered dictionary mapping an attribute to the column of values for each ent.

In [23]:
doc = doc_consumer(doc)

In [24]:
doc._.data

{'ent': OrderedDict([('text',
               ['Hydrochlorothiazide',
                'chemo',
                'chemo',
                'Miconazole Nitrate',
                '2',
                'Powder',
                'One (1)',
                'Appl',
                'Topical',
                'BID',
                'Heparin Sodium (Porcine)',
                '5,000 unit/mL',
                'Solution',
                'One (1)',
                'Injection',
                'TID (3 times a day)',
                'Acetaminophen',
                '160 mg/5 mL',
                'Elixir',
                'One (1)',
                'PO']),
              ('start_char',
               [163,
                556,
                652,
                1183,
                1202,
                1206,
                1218,
                1226,
                1231,
                1240,
                1274,
                1299,
                1313,
                1327,
                1335

If you have pandas installed, you can also now convert your doc to a DataFrame for each of the four levels:

In [25]:
# !pip install pandas

In [26]:
doc._.to_dataframe("ent").head(10)

,text,start_char,end_char,label_,is_negated,is_uncertain,is_historical,is_hypothetical,is_family,section_category,section_parent
0,Hydrochlorothiazide,163,182,DRUG,False,False,False,False,False,allergies,None
1,chemo,556,561,DRUG,True,False,False,False,False,history_of_present_illness,None
2,chemo,652,657,DRUG,False,False,False,False,False,past_medical_history,None
3,Miconazole Nitrate,1183,1201,DRUG,False,False,False,False,False,medications,None
4,2,1202,1203,STRENGTH,False,False,False,False,False,medications,None
5,Powder,1206,1212,FORM,False,False,False,False,False,medications,None
6,One (1),1218,1225,DOSAGE,False,False,False,False,False,medications,None
7,Appl,1226,1230,FORM,False,False,False,False,False,medications,None
8,Topical,1231,1238,ROUTE,False,False,False,False,False,medications,None
9,BID,1240,1243,FREQUENCY,False,False,False,False,False,medications,None


Now let's go through each of the 4 levels individually. You can specify a single level of data by either passing in the dtype to `doc._.get_data(dtype)` or accessing individual attributes.

## Ents data

In [27]:
ent_data = doc._.ent_data

In [28]:
ent_data.keys()

odict_keys(['text', 'start_char', 'end_char', 'label_', 'is_negated', 'is_uncertain', 'is_historical', 'is_hypothetical', 'is_family', 'section_category', 'section_parent'])

You can also access the data row-wise rather then column-wise:

In [29]:
doc._.get_data("ent", as_rows=True)[:2]

[('Hydrochlorothiazide',
  163,
  182,
  'DRUG',
  False,
  False,
  False,
  False,
  False,
  'allergies',
  None),
 ('chemo',
  556,
  561,
  'DRUG',
  True,
  False,
  False,
  False,
  False,
  'history_of_present_illness',
  None)]

In [30]:
ents_df = doc._.to_dataframe("ent")

In [31]:
ents_df.head()

,text,start_char,end_char,label_,is_negated,is_uncertain,is_historical,is_hypothetical,is_family,section_category,section_parent
0,Hydrochlorothiazide,163,182,DRUG,False,False,False,False,False,allergies,None
1,chemo,556,561,DRUG,True,False,False,False,False,history_of_present_illness,None
2,chemo,652,657,DRUG,False,False,False,False,False,past_medical_history,None
3,Miconazole Nitrate,1183,1201,DRUG,False,False,False,False,False,medications,None
4,2,1202,1203,STRENGTH,False,False,False,False,False,medications,None


In [33]:
ents_df[ents_df["is_negated"] == True]

,text,start_char,end_char,label_,is_negated,is_uncertain,is_historical,is_hypothetical,is_family,section_category,section_parent
1,chemo,556,561,DRUG,True,False,False,False,False,history_of_present_illness,None


## Section data

In [34]:
section_data = doc._.section_data

In [35]:
section_data.keys()

odict_keys(['section_category', 'section_title_text', 'section_title_start_char', 'section_title_end_char', 'section_text', 'section_text_start_char', 'section_text_end_char', 'section_parent'])

In [36]:
doc._.get_data("section", as_rows=True)[0]

(None,
 None,
 0,
 0,
 'Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]\n\nDate of Birth:  [**2498-8-19**]             Sex:   F\n\n',
 0,
 134,
 None)

In [37]:
section_df = doc._.to_dataframe("section")

In [38]:
section_df.head()

,section_category,section_title_text,section_title_start_char,section_title_end_char,section_text,section_text_start_char,section_text_end_char,section_parent
0,None,None,0,0,Admission Date: [**2573-5-30**] ...,0,134,None
1,other,Service:,134,142,Service: SURGERY\n\n,134,152,None
2,allergies,Allergies:,152,162,Allergies:\nHydrochlorothiazide\n\nAttending:[...,152,222,None
3,chief_complaint,Chief Complaint:,222,238,Chief Complaint:\nAbdominal pain\n\nMajor Surg...,222,350,None
4,history_of_present_illness,History of Present Illness:,350,377,History of Present Illness:\n74y female with t...,350,564,None


## Context data

In [39]:
context_data = doc._.context_data

In [40]:
context_data.keys()

odict_keys(['ent_text', 'ent_label_', 'ent_start_char', 'ent_end_char', 'modifier_text', 'modifier_category', 'modifier_direction', 'modifier_start_char', 'modifier_end_char', 'modifier_scope_start_char', 'modifier_scope_end_char'])

In [41]:
doc._.get_data("context", as_rows=True)[:2]

[('chemo',
  'DRUG',
  556,
  561,
  'is not',
  'NEGATED_EXISTENCE',
  'FORWARD',
  535,
  541,
  542,
  541)]

In [42]:
context_df = doc._.to_dataframe("context")

In [43]:
context_df.head()

,ent_text,ent_label_,ent_start_char,ent_end_char,modifier_text,modifier_category,modifier_direction,modifier_start_char,modifier_end_char,modifier_scope_start_char,modifier_scope_end_char
0,chemo,DRUG,556,561,is not,NEGATED_EXISTENCE,FORWARD,535,541,542,541


## Doc

In [44]:
doc_data = doc._.doc_data

In [45]:
doc_data.keys()

odict_keys(['text'])

In [46]:
doc_df = doc._.to_dataframe("doc")

In [47]:
doc_df

,text
0,Admission Date: [**2573-5-30**] ...


## Customizing attributes
You can customize the values in `dtype_attrs` to modify what attributes are stored. "doc" and "ent" dtypes can take additional attributes which aren't included in the default, but "section" and "context" attributes can only take subsets of the defaults.

You can see the default values by calling the class method below:

In [48]:
DocConsumer.get_default_attrs()

{'ent': ['text',
  'start_char',
  'end_char',
  'label_',
  'is_negated',
  'is_uncertain',
  'is_historical',
  'is_hypothetical',
  'is_family',
  'section_category',
  'section_parent'],
 'section': ['section_category',
  'section_title_text',
  'section_title_start_char',
  'section_title_end_char',
  'section_text',
  'section_text_start_char',
  'section_text_end_char',
  'section_parent'],
 'context': ['ent_text',
  'ent_label_',
  'ent_start_char',
  'ent_end_char',
  'modifier_text',
  'modifier_category',
  'modifier_direction',
  'modifier_start_char',
  'modifier_end_char',
  'modifier_scope_start_char',
  'modifier_scope_end_char'],
 'doc': ['text']}

Let's create a second model here and add a second `DocConsumer` with customized attributes. We'll also add a new custom attribute for `Doc`, `report_id`, and include it.

In [50]:
from spacy.tokens import Doc
Doc.set_extension("report_title", default="")

In [62]:
doc2 = nlp("Previously on antibiotics.")

In [63]:
doc2._.report_title = "example_document"

In [67]:
doc_consumer2 = DocConsumer(nlp, dtypes=("ent", "doc"), 
                            dtype_attrs={
                                "ent": [
                                    "text",
                                    "label_",
                                    "is_negated",
                                    "section_category",
                                    
                                ],
                                "doc":
                                ["text", "report_title"]
                                
                            }
)

In [68]:
doc_consumer2(doc2)

Previously on antibiotics.

In [69]:
doc2._.to_dataframe("ent")

,text,label_,is_negated,section_category
0,antibiotics,DRUG,False,None


In [70]:
doc2._.to_dataframe("doc")

,text,report_title
0,Previously on antibiotics.,example_document


# Writer and Reader
The reader and writer classes are utilities for I/O. Here we'll show how to use it to connect to a sqlite database, read in text, and write them back to a new table.

In [71]:
from medspacy.io import DbWriter, DbReader, DbConnect

## DbConn
DbConnect is a wrapper for either a pyodbc or sqlite3 connection. It can then be passed into the DbReader and DbWriter classes to retrieve/store document data.

You can pass in either information for a pyodbc connection string or directly pass in a sqlite or pyodbc connection object. Here, we'll pass in a connection to our sqlite database.

In [72]:
sq3_conn = sqlite3.connect("./medspacy_demo.db")

In [73]:
cursor = sq3_conn.cursor()

In [74]:
conn = DbConnect(conn=sq3_conn)

Opened connection to None.None


## DbReader
DbReader is a utility fo reading docs from a database. It takes two arguments:
- **db_conn**: A `DbConnect` object
- **read_query**: The SQL query to read documents

Calling `reader.read()` will then return the results of the specified query.

In [75]:
read_query = """
SELECT text
FROM texts
"""

In [76]:
reader = DbReader(conn, read_query)

In [77]:
rslts = reader.read()

Read 2 rows with query: 
SELECT text
FROM texts



In [78]:
texts = [r[0] for r in rslts]

In [79]:
print(texts[0][:100])

Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]

Date of Birth:  [**2


In [80]:
print(texts[1])

There is no evidence of pneumonia.


#### Now we will process these docs and prepare to write them back:

In [84]:
nlp.add_pipe("medspacy_doc_consumer");

In [85]:
docs = list(nlp.pipe(texts))

## DbWriter
DbWriter is a utility class for writing structured data back to a database. Here we'll use it to store out processed doc to a new table called `ents`.

DbWriter takes the following arguments:

- **destination_table**: The name of the table to insert values into
- **cols**: Column names of the destination table
- **col_types**: SQL data types of the columns
- **doc_dtype**: DocConsumer data type, default 'ent'
- **create_table**: Whether or not to create the table if it does not exist, default False
- **drop_existing**: Whether to drop the table if it does exist, default False,
- **write_batch_size**: Default 100


Our column names will be the "ents" attributes in our consumer:

In [86]:
doc_consumer.dtype_attrs["ent"]

['text',
 'start_char',
 'end_char',
 'label_',
 'is_negated',
 'is_uncertain',
 'is_historical',
 'is_hypothetical',
 'is_family',
 'section_category',
 'section_parent']

Now we'll define the SQL datatypes for each column:

In [87]:
col_types = [
    "varchar(1000)",
    "int",
    "int",
    "varchar(100)",
    "int",
    "int",
    "int",
    "int",
    "int",
    "varchar(100)",
    "varchar(100)"
]

In [88]:
for (name, typ) in zip(doc_consumer.dtype_attrs["ent"], col_types):
    print(name, typ)

text varchar(1000)
start_char int
end_char int
label_ varchar(100)
is_negated int
is_uncertain int
is_historical int
is_hypothetical int
is_family int
section_category varchar(100)
section_parent varchar(100)


No we'll instantiate our writer and write the doc's entities to a new table:

In [89]:
writer = DbWriter(conn, "ents", cols=doc_consumer.dtype_attrs["ent"], 
                  col_types=col_types,
                  doc_dtype="ent",
                 create_table=True, drop_existing=True)

Created table ents with query: CREATE TABLE ents (text varchar(1000), start_char int, end_char int, label_ varchar(100), is_negated int, is_uncertain int, is_historical int, is_hypothetical int, is_family int, section_category varchar(100), section_parent varchar(100))


In [90]:
for doc in docs:
    writer.write(doc)

Wrote 21 rows with query: INSERT INTO ents (text, start_char, end_char, label_, is_negated, is_uncertain, is_historical, is_hypothetical, is_family, section_category, section_parent) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
Wrote 0 rows with query: INSERT INTO ents (text, start_char, end_char, label_, is_negated, is_uncertain, is_historical, is_hypothetical, is_family, section_category, section_parent) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)


Now we can query our table to retrieve the data we wrote:

In [91]:
query = """
SELECT *
FROM ents
LIMIT 2;
"""

In [92]:
cursor.execute(query)

In [93]:
cursor.fetchall()

[('Hydrochlorothiazide', 163, 182, 'DRUG', 0, 0, 0, 0, 0, 'allergies', None),
 ('chemo',
  556,
  561,
  'DRUG',
  1,
  0,
  0,
  0,
  0,
  'history_of_present_illness',
  None)]

In [94]:
query = """
SELECT label_, COUNT(1)
FROM ents
GROUP BY label_
"""

In [95]:
cursor.execute(query)

In [96]:
cursor.fetchall()

[('DOSAGE', 3),
 ('DRUG', 6),
 ('FORM', 4),
 ('FREQUENCY', 2),
 ('ROUTE', 3),
 ('STRENGTH', 3)]

## Pipeline
Once we have our reader and writer objects, we can pass them in to a `Pipeline`, which will read texts in batches, process them with NLP, and write them back to the database as specified by the writer.

`Pipeline` takes the following arguments:
- **reader: A DbReader object**
- **writer: A Dbwriter object**
- **nlp: A spaCy model**
- **dtype: The DocConsumer data type to write to a database. Default "ent**

In order for the pipeline to work, the first column of both the source and destination tables must some sort of document identifier (ie., "text_id"). 

In [97]:
from medspacy.io import Pipeline

In [98]:
sq3_conn_read = sqlite3.connect("./medspacy_demo.db")
conn = DbConnect(conn=sq3_conn_read)

Opened connection to None.None


In [99]:
reader = DbReader(conn, read_query="SELECT text_id, text FROM texts")
writer = DbWriter(conn, "ents", 
                  # Add the text_id to the column names and types
                  cols=["text_id"]+doc_consumer.dtype_attrs["ent"], 
                  col_types=["int"] + col_types,
                  doc_dtype="ent",
                 create_table=True, drop_existing=True)

Created table ents with query: CREATE TABLE ents (text_id int, text varchar(1000), start_char int, end_char int, label_ varchar(100), is_negated int, is_uncertain int, is_historical int, is_hypothetical int, is_family int, section_category varchar(100), section_parent varchar(100))


In [102]:
pipeline = Pipeline(nlp, reader, writer, nlp, dtype="ent")

In [103]:
pipeline.process()

Read 2 rows with query: SELECT text_id, text FROM texts
Wrote 21 rows with query: INSERT INTO ents (text_id, text, start_char, end_char, label_, is_negated, is_uncertain, is_historical, is_hypothetical, is_family, section_category, section_parent) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
Connection closed.


In [104]:
conn = sqlite3.connect("./medspacy_demo.db")
cursor = conn.cursor()

In [105]:
cursor.execute("""
SELECT text_id, label_, COUNT(1)
FROM ents
GROUP BY text_id, label_
""")

In [106]:
cursor.fetchall()

[(1, 'DOSAGE', 3),
 (1, 'DRUG', 6),
 (1, 'FORM', 4),
 (1, 'FREQUENCY', 2),
 (1, 'ROUTE', 3),
 (1, 'STRENGTH', 3)]